In [2]:
import sqlite3
import pandas as pd

# Nome do arquivo de texto que você deseja ler
caminho_arquivo_texto = 'PNADC_022023.txt'

# Nome do banco de dados SQLite
nome_banco_de_dados = 'pnad.db'

# Nome da tabela no banco de dados
nome_tabela = 'pnad'

# Conecte-se ao banco de dados (ou crie-o se não existir)
conn = sqlite3.connect(nome_banco_de_dados)
cursor = conn.cursor()

# Crie uma tabela no banco de dados com uma coluna para os dados do arquivo de texto
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {nome_tabela} (
        coluna_texto TEXT
    )
''')

# Abra o arquivo de texto e leia todo o seu conteúdo
with open(caminho_arquivo_texto, 'r') as arquivo_texto:
    linhas = arquivo_texto.readlines()

# Divida as linhas em partes menores (por exemplo, 1000 caracteres por parte)
partes = [''.join(linhas[i:i+1000]) for i in range(0, len(linhas), 1000)]

# Insira cada parte na tabela
for parte in partes:
    cursor.execute(f'INSERT INTO {nome_tabela} (coluna_texto) VALUES (?)', (parte,))

# Commit das alterações
conn.commit()

# Feche a conexão com o banco de dados
conn.close()

# Crie um DataFrame com uma única coluna chamada "conteudo"
df_conteudo = pd.DataFrame({'conteudo': linhas})


In [7]:
#print(df_conteudo.head(20))
print(df_conteudo.tail(1))

                                                 conteudo
474574  202325353  53006088053101111411111000293.67656...


In [ ]:
# Criação da base separada em função do index, que contem o mapa do tamanho das variaveis e seu nome 

# Subida do index que mapea os dados do pnad
df_index = pd.read_excel('index.xlsx')
df_index = pd.DataFrame(df_index)

# Crie um novo DataFrame para armazenar os resultados
df_resultado = pd.DataFrame()

# Itere pelo df_index
for _, row in df_index.iterrows():
    posicao = row['Position']
    tamanho = row['Size']
    variavel = row['Variables']
    
    # Use as informações de posição e tamanho para dividir a string em novas colunas
    df_conteudo[variavel] = df_conteudo['conteudo'].str.slice(posicao - 1, posicao - 1 + tamanho)

# Remova a coluna 'conteudo' se necessário
df_resultado = df_conteudo.drop(columns=['conteudo'])

In [9]:
# Salve o DataFrame df_dividido em um arquivo csv
df_resultado.to_csv('pnead_dividido2.csv', index=False)